# Projeto de Precificação de Imóveis de São Paulo

## Etapa 1: Extração e Tratamento dos dados

Na primeira etapa do projeto, será realizado a extração e tratamento das bases de dados dos imóveis do município de São Paulo de modo a criar uma base de dados consistente para ser utilizada na etapa de análise exploratória, são eles:

* Dados do valor dos imovéis (Fonte: Kaggle)
* Dados do censo imobiliário (Fonte: IBGE)
* Dados dos endereços 
* Dados geográficos (Fonte: IBGE)

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
from shapely.geometry import Point, Polygon

## Tratamento

### Dados do valor dos imóveis

In [3]:
path_aula = 'https://gist.githubusercontent.com/tgcsantos/3bdb29eba6ce391e90df2b72205ba891/raw/22fa920e80c9fa209a9fccc8b52d74cc95d1599b/dados_imoveis.csv'

In [4]:
path = "https://raw.githubusercontent.com/rafaelladuarte/precificacao_imoveis/main/data_properties_SP.csv"
df_imovel = pd.read_csv(path)
df_imovel.sample(5)

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price
5294,"Cidade Jardim, São Paulo",SP,970,4,6,6,R$ 45.000\n /Mês
6735,"Instituto de Previdência, São Paulo",SP,120,3,1,2,R$ 2.600\n /Ano
7496,Rua Antônio Dario Menezes,"Vila Romano, São Paulo",141,3,3,4,R$ 800.000
1350,Rua Francisco Xavier da Rocha,"Parque Alves de Lima, São Paulo",160,3,4,4,R$ 790.000
1052,"Vila Sílvia, São Paulo",SP,230,5,3,2,R$ 590.000


Limpando os espaços vazios extras nos valores dos campos

In [5]:
df_imovel = df_imovel.applymap(
    lambda x: x.strip() if isinstance(x, str) else x
)

Nota-se que o dataframe não possui a coluna que nos diz a respeito em qual bairro da cidade de São Paulo o imóvel localiza. No entanto, é possivel observar que essa informação pode ser encontrada na coluna Street ou City.

Ao separar os valores do campo Street por virgula, nota-se que pode ocorrer 3 partições

In [6]:
df_imovel["Street"].str.split(pat=",",expand = True)

,0,1,2
0,Avenida Itacira,255,None
1,Rua Aurelia Perez Alvarez,42,None
2,Rua Alba Valdez,None,None
3,Jardim Morumbi,São Paulo,None
4,Rua Tobias Barreto,195,None
...,...,...,...
10028,Rua Tapuçu,None,None
10029,Rua Guararema,None,None
10030,Rua Estero Belaco,None,None
10031,Rua Manuel Onha,,514


Inserindo as partições da separação no dataframe

In [7]:
df_imovel[
    [
        "Street_1",
        "Street_2", 
        "Street_3"
    ]
]  = df_imovel["Street"].str.split(pat=",",expand = True)

Visualizando as partições no dataframe

In [8]:
df_imovel.sample(5)

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price,Street_1,Street_2,Street_3
6138,"Rua Emboabas, 50","Brooklin Paulista, São Paulo",418,4,5,6,R$ 2.700.000,Rua Emboabas,50,None
6856,"Bosque da Saúde, São Paulo",SP,486,3,1,1,R$ 1.900.000,Bosque da Saúde,São Paulo,None
4780,Rua Alexandre Buairide,"Vila Mariana, São Paulo",270,4,5,3,R$ 1.600.000,Rua Alexandre Buairide,None,None
5814,Rua do Piquete,"Vila Monumento, São Paulo",110,3,3,2,R$ 750.000,Rua do Piquete,None,None
9310,Rua Búzios,"Jardim Franca, São Paulo",54,2,3,1,R$ 375.000,Rua Búzios,None,None


Já ao separar os valores do campo City por virgula, ocorrer apenas 2 partições

In [9]:
df_imovel["City"].str.split(pat=",",expand = True)

,0,1
0,Planalto Paulista,São Paulo
1,Jardim dos Estados,São Paulo
2,Jardim Reimberg,São Paulo
3,SP,None
4,Mooca,São Paulo
...,...,...
10028,Vila Sofia,São Paulo
10029,Bosque da Saúde,São Paulo
10030,Vila da Saúde,São Paulo
10031,Vila Oratório,São Paulo


Inserindo as partições da separação no dataframe

In [10]:
df_imovel[["City_1", "City_2"]]  = df_imovel["City"].str.split(pat=",",expand = True)

Visualizando as partições no dataframe

In [11]:
df_imovel.sample(5)

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price,Street_1,Street_2,Street_3,City_1,City_2
2481,"Jardim Leonor, São Paulo",SP,491,4,8,4,R$ 3.100.000,Jardim Leonor,São Paulo,None,SP,None
356,Rua José Ubaldo Lomônaco,"Jardim da Glória, São Paulo",430,5,6,4,R$ 3.150.000,Rua José Ubaldo Lomônaco,None,None,Jardim da Glória,São Paulo
5259,Rua Professor Rocca Dordal,"Vila Pompéia, São Paulo",320,2,1,2,R$ 2.240.000,Rua Professor Rocca Dordal,None,None,Vila Pompéia,São Paulo
8143,"Rua Rodrigues Guião, 146","Pinheiros, São Paulo",130,3,3,1,R$ 2.000.000,Rua Rodrigues Guião,146,None,Pinheiros,São Paulo
70,"Rua Major Alfredo Romão, 92","Vila Clarice, São Paulo",99,3,2,2,R$ 650.000,Rua Major Alfredo Romão,92,None,Vila Clarice,São Paulo


Atualizando o campo City para São Paulo, cidade dos dados que está sendo analisado

In [12]:
df_imovel["City"] = "São Paulo"

In [13]:
df_imovel.sample(2)

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price,Street_1,Street_2,Street_3,City_1,City_2
2068,"Morumbi, São Paulo",São Paulo,396,4,4,11,R$ 2.000.000,Morumbi,São Paulo,None,SP,None
4877,Rua Marcelino Champagnat,São Paulo,237,4,4,3,R$ 1.650.000,Rua Marcelino Champagnat,None,None,Jardim da Glória,São Paulo


Criando a coluna Bairro

In [14]:
df_imovel["Bairro"] = df_imovel["City_1"]

Removendo valores do campo Street, quando a informação representa um bairro

In [15]:
df_imovel.loc[
    df_imovel["Bairro"] == "SP", "Street"
] = np.nan

Inserindo o valor do Bairro na sua respectiva coluna

In [16]:
df_imovel.loc[
    df_imovel["Bairro"] == "SP", "Bairro"
] = df_imovel['Street_1']

In [17]:
df_imovel.head()

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price,Street_1,Street_2,Street_3,City_1,City_2,Bairro
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,R$ 7.000.000,Avenida Itacira,255,None,Planalto Paulista,São Paulo,Planalto Paulista
1,"Rua Aurelia Perez Alvarez, 42",São Paulo,469-524,3-6,3-6,4,R$ 3.700.000,Rua Aurelia Perez Alvarez,42,None,Jardim dos Estados,São Paulo,Jardim dos Estados
2,Rua Alba Valdez,São Paulo,125,4,3,2,R$ 380.000,Rua Alba Valdez,None,None,Jardim Reimberg,São Paulo,Jardim Reimberg
3,NaN,São Paulo,310,3,2,4,R$ 685.000,Jardim Morumbi,São Paulo,None,SP,None,Jardim Morumbi
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,R$ 540.000,Rua Tobias Barreto,195,None,Mooca,São Paulo,Mooca


Removendo as colunas desnecessarias

In [18]:
df_imovel = df_imovel.drop(
    columns = [
        'Street_1',
        'Street_2',	
        'Street_3',
        'City_1',
        'City_2',
    ]
)
df_imovel.head()

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price,Bairro
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,R$ 7.000.000,Planalto Paulista
1,"Rua Aurelia Perez Alvarez, 42",São Paulo,469-524,3-6,3-6,4,R$ 3.700.000,Jardim dos Estados
2,Rua Alba Valdez,São Paulo,125,4,3,2,R$ 380.000,Jardim Reimberg
3,NaN,São Paulo,310,3,2,4,R$ 685.000,Jardim Morumbi
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,R$ 540.000,Mooca


In [19]:
df_imovel.columns = ['Rua', 'cidade','metragem','quartos','banheiros','vagas','preco','bairro']

In [20]:
df_imovel.head()

,Rua,cidade,metragem,quartos,banheiros,vagas,preco,bairro
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,R$ 7.000.000,Planalto Paulista
1,"Rua Aurelia Perez Alvarez, 42",São Paulo,469-524,3-6,3-6,4,R$ 3.700.000,Jardim dos Estados
2,Rua Alba Valdez,São Paulo,125,4,3,2,R$ 380.000,Jardim Reimberg
3,NaN,São Paulo,310,3,2,4,R$ 685.000,Jardim Morumbi
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,R$ 540.000,Mooca


Tratando os valores da coluna preço

Nota-se que elas estão no tipo string

In [21]:
type(df_imovel["preco"][0])

str

Separando a string por espaço

In [22]:
df_imovel["preco"].str.split(expand = True)

,0,1,2
0,R$,7.000.000,None
1,R$,3.700.000,None
2,R$,380.000,None
3,R$,685.000,None
4,R$,540.000,None
...,...,...,...
10028,R$,665.000,None
10029,R$,2.300.000,None
10030,R$,1.050.000,None
10031,R$,2.200,/Mês


In [23]:
df_imovel["preco"].str.split(expand = True)[2].unique()

array([None, '/Mês', '/Ano', '/Dia'], dtype=object)

In [24]:
df_imovel[["moeda", "preco_anuncio", "tipo_anuncio"]] = df_imovel["preco"].str.split(expand = True)

In [25]:
df_imovel.sample(3)

,Rua,cidade,metragem,quartos,banheiros,vagas,preco,bairro,moeda,preco_anuncio,tipo_anuncio
9949,Rua Aquino Fonseca,São Paulo,96,3,2,2,R$ 553.000,Jardim das Carmelitas,R$,553.000,None
9290,"Rua Cuiabá, , 758",São Paulo,160,2,1,2,R$ 800.000,Alto da Mooca,R$,800.000,None
676,Rua Corgie Assad Abdalla,São Paulo,529,4,5,6,R$ 2.000.000,Vila Sônia,R$,2.000.000,None


In [26]:
df_imovel[df_imovel["tipo_anuncio"].isnull()]["tipo_anuncio"].unique()

array([None], dtype=object)

In [27]:
df_imovel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10033 entries, 0 to 10032
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rua            6596 non-null   object
 1   cidade         10033 non-null  object
 2   metragem       10033 non-null  object
 3   quartos        10033 non-null  object
 4   banheiros      10033 non-null  object
 5   vagas          10033 non-null  object
 6   preco          10033 non-null  object
 7   bairro         10033 non-null  object
 8   moeda          10033 non-null  object
 9   preco_anuncio  10033 non-null  object
 10  tipo_anuncio   852 non-null    object
dtypes: object(11)
memory usage: 862.3+ KB


In [28]:
df_imovel = df_imovel[df_imovel["tipo_anuncio"].isnull()]
df_imovel

,Rua,cidade,metragem,quartos,banheiros,vagas,preco,bairro,moeda,preco_anuncio,tipo_anuncio
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,R$ 7.000.000,Planalto Paulista,R$,7.000.000,None
1,"Rua Aurelia Perez Alvarez, 42",São Paulo,469-524,3-6,3-6,4,R$ 3.700.000,Jardim dos Estados,R$,3.700.000,None
2,Rua Alba Valdez,São Paulo,125,4,3,2,R$ 380.000,Jardim Reimberg,R$,380.000,None
3,NaN,São Paulo,310,3,2,4,R$ 685.000,Jardim Morumbi,R$,685.000,None
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,R$ 540.000,Mooca,R$,540.000,None
...,...,...,...,...,...,...,...,...,...,...,...
10027,Avenida Odila,São Paulo,262,3,6,4,R$ 1.500.000,Planalto Paulista,R$,1.500.000,None
10028,Rua Tapuçu,São Paulo,90,2,2,2,R$ 665.000,Vila Sofia,R$,665.000,None
10029,Rua Guararema,São Paulo,600,8,7,5,R$ 2.300.000,Bosque da Saúde,R$,2.300.000,None
10030,Rua Estero Belaco,São Paulo,200,3,3,6,R$ 1.050.000,Vila da Saúde,R$,1.050.000,None


In [29]:
df_imovel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9181 entries, 0 to 10032
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rua            6129 non-null   object
 1   cidade         9181 non-null   object
 2   metragem       9181 non-null   object
 3   quartos        9181 non-null   object
 4   banheiros      9181 non-null   object
 5   vagas          9181 non-null   object
 6   preco          9181 non-null   object
 7   bairro         9181 non-null   object
 8   moeda          9181 non-null   object
 9   preco_anuncio  9181 non-null   object
 10  tipo_anuncio   0 non-null      object
dtypes: object(11)
memory usage: 860.7+ KB


Substituindo o ponto dos milhares para vazio, pra conseguir converter os valores para float

In [30]:
df_imovel["preco_anuncio"].str.replace(".","",regex=False).astype(float)

0        7000000.0
1        3700000.0
2         380000.0
3         685000.0
4         540000.0
           ...    
10027    1500000.0
10028     665000.0
10029    2300000.0
10030    1050000.0
10032     270000.0
Name: preco_anuncio, Length: 9181, dtype: float64

Adicionando uma coluna no dataframe para o preço do anuncio em formato float

In [31]:
df_imovel["preco_anuncio_float"] = df_imovel["preco_anuncio"].str.replace(".","",regex=True).astype(float)

In [32]:
df_imovel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9181 entries, 0 to 10032
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Rua                  6129 non-null   object 
 1   cidade               9181 non-null   object 
 2   metragem             9181 non-null   object 
 3   quartos              9181 non-null   object 
 4   banheiros            9181 non-null   object 
 5   vagas                9181 non-null   object 
 6   preco                9181 non-null   object 
 7   bairro               9181 non-null   object 
 8   moeda                9181 non-null   object 
 9   preco_anuncio        9181 non-null   object 
 10  tipo_anuncio         0 non-null      object 
 11  preco_anuncio_float  9181 non-null   float64
dtypes: float64(1), object(11)
memory usage: 932.4+ KB


In [33]:
df_imovel.sample(5)

,Rua,cidade,metragem,quartos,banheiros,vagas,preco,bairro,moeda,preco_anuncio,tipo_anuncio,preco_anuncio_float
9526,Rua Rio do Peixe,São Paulo,107,3,3,2,R$ 720.000,Vila Lúcia,R$,720.000,None,720000.0
364,NaN,São Paulo,200,2,3,4,R$ 1.090.000,Jardim Virginia Bianca,R$,1.090.000,None,1090000.0
5665,"Rua Blumenau, 105",São Paulo,125,3,4,2,R$ 695.000,Vila Leopoldina,R$,695.000,None,695000.0
1023,Rua Carangola,São Paulo,800,4,6,8,R$ 6.100.000,Jardim Guedala,R$,6.100.000,None,6100000.0
4279,Rua Carlos Tiago Pereira,São Paulo,113,3,3,3,R$ 790.000,Bosque da Saúde,R$,790.000,None,790000.0


Removendo as colunas desnecessarias

In [34]:
df_imovel = df_imovel.drop(
    columns = [
        'tipo_anuncio',
        'moeda',
        'preco_anuncio',
        'preco'
    ]
)

In [35]:
df_imovel['metragem']

0           1000
1        469-524
2            125
3            310
4            100
          ...   
10027        262
10028         90
10029        600
10030        200
10032        117
Name: metragem, Length: 9181, dtype: object

Substituindo os valores que não são numeros nas colunas de metragem e vagas para NaN que significa Not a Number

In [36]:
df_imovel['metragem'] = df_imovel['metragem'].apply(
    lambda x: np.nan if '-' in x else x
)
df_imovel['metragem']

0        1000
1         NaN
2         125
3         310
4         100
         ... 
10027     262
10028      90
10029     600
10030     200
10032     117
Name: metragem, Length: 9181, dtype: object

In [37]:
df_imovel['vagas'] = df_imovel['vagas'].apply(
    lambda x: np.nan if '-' in x else x
)
df_imovel['vagas']

0        6
1        4
2        2
3        4
4        2
        ..
10027    4
10028    2
10029    5
10030    6
10032    2
Name: vagas, Length: 9181, dtype: object

Removendo as linhas que possuem valores nulos nas colunas de metragem e vagas

In [38]:
df_imovel = df_imovel.dropna(subset=["metragem","vagas"])
df_imovel

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,Planalto Paulista,7000000.0
2,Rua Alba Valdez,São Paulo,125,4,3,2,Jardim Reimberg,380000.0
3,NaN,São Paulo,310,3,2,4,Jardim Morumbi,685000.0
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,Mooca,540000.0
5,Rua Graham Bell,São Paulo,440,4,4,6,Santo Amaro,1980000.0
...,...,...,...,...,...,...,...,...
10027,Avenida Odila,São Paulo,262,3,6,4,Planalto Paulista,1500000.0
10028,Rua Tapuçu,São Paulo,90,2,2,2,Vila Sofia,665000.0
10029,Rua Guararema,São Paulo,600,8,7,5,Bosque da Saúde,2300000.0
10030,Rua Estero Belaco,São Paulo,200,3,3,6,Vila da Saúde,1050000.0


In [39]:
df_imovel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9177 entries, 0 to 10032
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Rua                  6125 non-null   object 
 1   cidade               9177 non-null   object 
 2   metragem             9177 non-null   object 
 3   quartos              9177 non-null   object 
 4   banheiros            9177 non-null   object 
 5   vagas                9177 non-null   object 
 6   bairro               9177 non-null   object 
 7   preco_anuncio_float  9177 non-null   float64
dtypes: float64(1), object(7)
memory usage: 645.3+ KB


Mudando o tipo dos valores das colunas metragem, quartos, banheiros e vagas para inteiro

In [40]:
df_imovel["metragem"] = df_imovel["metragem"].astype(int)
df_imovel["quartos"] = df_imovel["quartos"].astype(int)
df_imovel["banheiros"] = df_imovel["banheiros"].astype(int)
df_imovel["vagas"] = df_imovel["vagas"].astype(int)

In [41]:
df_imovel.sample(5)

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float
833,Rua Comendador Bichara Moherdauí,São Paulo,1000,3,4,4,Jardim Leonor,5500000.0
4531,Rua Suécia,São Paulo,700,6,6,8,Jardim Europa,10900000.0
4595,NaN,São Paulo,700,5,8,6,Jardim América,14500000.0
9182,NaN,São Paulo,97,1,1,2,Itaquera,304500.0
7719,Rua Comandante Ismael Guilherme,São Paulo,600,3,6,4,Jardim Luzitânia,12000000.0


Removendo caracter especial do campo Rua

In [42]:
df_imovel["apenas_rua"] = df_imovel["Rua"].str.extract(r'(^[\w ]+)')
df_imovel["apenas_rua"] = df_imovel["apenas_rua"].str.lower().str.strip()
df_imovel.head()

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float,apenas_rua
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,Planalto Paulista,7000000.0,avenida itacira
2,Rua Alba Valdez,São Paulo,125,4,3,2,Jardim Reimberg,380000.0,rua alba valdez
3,NaN,São Paulo,310,3,2,4,Jardim Morumbi,685000.0,NaN
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,Mooca,540000.0,rua tobias barreto
5,Rua Graham Bell,São Paulo,440,4,4,6,Santo Amaro,1980000.0,rua graham bell


Exportando o dataframe dos valores dos imoveis para um arquivo csv

In [43]:
# df_imovel.to_csv('dados_imoveis_tratados.csv')

### Dados do setor imobiliário

Nessa base de dados encontramos os dados referentes ao censo do setor imobiliário, com o codigo de cada setor regional

In [44]:
path_setor = 'https://gist.githubusercontent.com/tgcsantos/85f8c7b0a2edbc3e27fcad619b37d886/raw/a4954781e6bca9cb804062a3eea0b3b84679daf4/Basico_SP1.csv'
df_setor = pd.read_csv(path_setor)
df_setor.sample(2)

,,,,,,,,,,Cod_setor;Cod_Grandes Regi�es;Nome_Grande_Regiao;Cod_UF;Nome_da_UF ;Cod_meso;Nome_da_meso;Cod_micro;Nome_da_micro;Cod_RM;Nome_da_RM;Cod_municipio;Nome_do_municipio;Cod_distrito;Nome_do_distrito;Cod_subdistrito;Nome_do_subdistrito;Cod_bairro;Nome_do_bairro;Situacao_setor;Tipo_setor;V001;V002;V003;V004;V005;V006;V007;V008;V009;V010;V011;V012;
"355030802000077;3;""Regi�o Sudeste"";35;""S�o Paulo"";3515;""Metropolitana de S�o Paulo"";35061;""S�o Paulo"";20;""RM S�o Paulo"";3550308;""S�O PAULO"";355030802;""ALTO DE PINHEIROS"";35503080200;""ALTO DE PINHEIROS"";3550308000;""S�O PAULO (todos os setores)"";1;0;173;314;1",82;,88;5312,6;152171819,74;6472,39;178068174,27;3766,39;95309603,93;5454,77;128990773,34.0
"355030863000128;3;""Regi�o Sudeste"";35;""S�o Paulo"";3515;""Metropolitana de S�o Paulo"";35061;""S�o Paulo"";20;""RM S�o Paulo"";3550308;""S�O PAULO"";355030863;""PIRITUBA"";35503086300;""PIRITUBA"";3550308000;""S�O PAULO (todos os setores)"";1;0;331;1033;3",12;2,63;1394,01;2383502,04;1507,9;2406573,23;890,43;1357374,73;1272,44;1453778,84.0


Nota-se que o tivemos problemas ao ler o arquivo, para resolver isso, primeiro determinamos o encoding para tratar os caracteres especiais, em seguida, determinamos o separador de colunas e delimitador de texto utilizado no arquivo.

In [45]:
df_setor = pd.read_csv(
    path_setor,
    encoding='ISO-8859-1',
    sep=';', thousands='.', decimal=','
)
df_setor.sample(5)

,Cod_setor,Cod_Grandes Regiï¿½es,Nome_Grande_Regiao,Cod_UF,Nome_da_UF,Cod_meso,Nome_da_meso,Cod_micro,Nome_da_micro,Cod_RM,...,V004,V005,V006,V007,V008,V009,V010,V011,V012,Unnamed: 33
10169,355030851000165,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,1.13,7814.76,2.400586e+08,9632.15,2.788317e+08,3164.89,86285665.03,6188.32,1.503629e+08,NaN
17751,355030894000016,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,2.45,1021.16,2.096999e+06,1548.94,2.366353e+06,733.45,1633406.15,1497.62,2.193720e+06,NaN
14573,355030876000077,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,2.01,817.95,1.084308e+06,1297.10,1.098067e+06,580.73,773725.90,1164.85,8.718467e+05,NaN
16745,355030888000016,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,3.90,838.44,2.616333e+05,932.65,2.028385e+05,541.06,262082.94,835.22,1.586575e+05,NaN
8989,355030845000164,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,0.74,4439.99,9.835008e+06,4571.87,9.522649e+06,3540.80,9515917.94,4013.67,8.886756e+06,NaN


In [46]:
df_setor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18363 entries, 0 to 18362
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Cod_setor              18363 non-null  int64  
 1   Cod_Grandes Regiï¿½es  18363 non-null  int64  
 2   Nome_Grande_Regiao     18363 non-null  object 
 3   Cod_UF                 18363 non-null  int64  
 4   Nome_da_UF             18363 non-null  object 
 5   Cod_meso               18363 non-null  int64  
 6   Nome_da_meso           18363 non-null  object 
 7   Cod_micro              18363 non-null  int64  
 8   Nome_da_micro          18363 non-null  object 
 9   Cod_RM                 18363 non-null  int64  
 10  Nome_da_RM             18363 non-null  object 
 11  Cod_municipio          18363 non-null  int64  
 12  Nome_do_municipio      18363 non-null  object 
 13  Cod_distrito           18363 non-null  int64  
 14  Nome_do_distrito       18363 non-null  object 
 15  Co

Quando vemos as informações gerais do dataframe gerado, vemos que ele possue muitos valores nulos. Desta forma, vamos trata-los removendo.

In [47]:
df_setor.dropna(how='all', axis=1, inplace=True)
df_setor.head()

,Cod_setor,Cod_Grandes Regiï¿½es,Nome_Grande_Regiao,Cod_UF,Nome_da_UF,Cod_meso,Nome_da_meso,Cod_micro,Nome_da_micro,Cod_RM,...,V003,V004,V005,V006,V007,V008,V009,V010,V011,V012
0,355030801000001,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,3.01,2.32,2091.32,8532451.78,2164.55,8673276.78,1227.41,4285771.99,1713.75,5152087.86
1,355030801000002,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,2.98,2.09,1650.09,3983441.31,1717.44,4030519.99,1045.78,2572133.32,1468.08,2991546.94
2,355030801000003,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,3.31,3.08,1280.43,5474593.06,2349.53,7553913.02,902.97,2735492.52,1786.79,3837674.74
3,355030801000004,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,3.16,2.26,1119.59,3572955.10,1859.13,4567289.50,869.09,2122409.02,1567.41,2735383.24
4,355030801000005,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,3.14,2.35,1416.90,2240207.58,1581.66,2240109.39,917.71,1570047.75,1356.35,1725877.85


### Dados dos endereços 

Nessa base encontramos os principais endereços existentes nos diversos estados do Brasil, cada um com sua geolocalização (latitude, longitude)

In [48]:
path_endereco = "/home/Rafaella.Duarte/Documentos/pasta_pessoal/precificacao_imoveis/Imersao_Dados/Large_Files/enderecos.csv"
df_endereco = pd.read_csv(path_endereco,low_memory=False)
df_endereco.sample(2)

,tipo_logr,logr_nome,logr_compl,numero,bairro,nome_localidade,sigla_uf,cep,latitude,longitude
903725,Rua,Comendador Ary Parreiras,Rua Comendador Ary Parreiras,NaN,Vila Recreio (Vila Inhomirim),Magé,RJ,25937830.0,-22.656602,-43.039834
822224,Rua,A,Rua A,NaN,Santo Antônio,Timon,MA,65630575.0,-5.098080,-42.833301


Como estamos analisando apenas os imovéis do municipio de São Paulo, em seguida iremos filtrar e seleciona apenas os dados desejados.

In [49]:
df_endereco = df_endereco.query("sigla_uf == 'SP'")
df_endereco.head()

,tipo_logr,logr_nome,logr_compl,numero,bairro,nome_localidade,sigla_uf,cep,latitude,longitude
1,Rua,João Antônio Vasconcelos,Rua João Antônio Vasconcelos - até 627/628,627,Amizade,Araçatuba,SP,16074320.0,-21.190276,-50.448686
413269,Rua,A,Rua A,NaN,Jardim Glória,Americana,SP,13468220.0,-22.756531,-47.343738
413270,Rua,A,Rua A,NaN,Jardim São Roque,Americana,SP,13469414.0,-22.743734,-47.348631
413271,Rua,A,Rua A,NaN,Iate Clube de Campinas,Americana,SP,13475620.0,-22.737846,-47.333569
413272,Rua,João Bandini,Rua João Bandini,NaN,Jardim Miriam,Americana,SP,13469072.0,-22.744283,-47.351588


Adicionando um coluna com o a rua dos endereços tratadas

In [50]:
df_endereco["rua"] = df_endereco["tipo_logr"] + " " + df_endereco["logr_nome"]
df_endereco["rua"] = df_endereco["rua"].str.lower().str.strip()
df_endereco.head()

,tipo_logr,logr_nome,logr_compl,numero,bairro,nome_localidade,sigla_uf,cep,latitude,longitude,rua
1,Rua,João Antônio Vasconcelos,Rua João Antônio Vasconcelos - até 627/628,627,Amizade,Araçatuba,SP,16074320.0,-21.190276,-50.448686,rua joão antônio vasconcelos
413269,Rua,A,Rua A,NaN,Jardim Glória,Americana,SP,13468220.0,-22.756531,-47.343738,rua a
413270,Rua,A,Rua A,NaN,Jardim São Roque,Americana,SP,13469414.0,-22.743734,-47.348631,rua a
413271,Rua,A,Rua A,NaN,Iate Clube de Campinas,Americana,SP,13475620.0,-22.737846,-47.333569,rua a
413272,Rua,João Bandini,Rua João Bandini,NaN,Jardim Miriam,Americana,SP,13469072.0,-22.744283,-47.351588,rua joão bandini


### Dados geoespaciais

Para ler dados geoespaciais da base do censitária do IBGE precisamos importar a biblioteca geopandas. Além disso, os dados precisam ser baixados localmente e estarem localizados no mesmo diretório (https://drive.google.com/drive/folders/1CycoanzYN2oxKHPO6zxvIQ6Og1LBcujv) 

In [51]:
path_geo = ('/home/Rafaella.Duarte/Documentos/pasta_pessoal/precificacao_imoveis/Imersao_Dados/Large_Files/35SEE250GC_SIR.shp')
df_geo = gpd.read_file(path_geo)
df_geo.head()

,ID,CD_GEOCODI,TIPO,CD_GEOCODS,NM_SUBDIST,CD_GEOCODD,NM_DISTRIT,CD_GEOCODM,NM_MUNICIP,NM_MICRO,NM_MESO,CD_GEOCODB,NM_BAIRRO,ID1,geometry
0,98237.0,354100005000009,URBANO,35410000500,None,354100005,PRAIA GRANDE,3541000,PRAIA GRANDE,SANTOS,METROPOLITANA DE SÃO PAULO,354100005001,Boqueirão,1,"POLYGON ((-46.41024 -24.01103, -46.41169 -24.0..."
1,98232.0,354100005000004,URBANO,35410000500,None,354100005,PRAIA GRANDE,3541000,PRAIA GRANDE,SANTOS,METROPOLITANA DE SÃO PAULO,354100005001,Boqueirão,2,"POLYGON ((-46.41606 -24.00245, -46.41598 -24.0..."
2,98230.0,354100005000002,URBANO,35410000500,None,354100005,PRAIA GRANDE,3541000,PRAIA GRANDE,SANTOS,METROPOLITANA DE SÃO PAULO,354100005001,Boqueirão,3,"POLYGON ((-46.41213 -24.00792, -46.41244 -24.0..."
3,98229.0,354100005000001,URBANO,35410000500,None,354100005,PRAIA GRANDE,3541000,PRAIA GRANDE,SANTOS,METROPOLITANA DE SÃO PAULO,354100005001,Boqueirão,4,"POLYGON ((-46.41134 -24.00552, -46.41196 -24.0..."
4,98231.0,354100005000003,URBANO,35410000500,None,354100005,PRAIA GRANDE,3541000,PRAIA GRANDE,SANTOS,METROPOLITANA DE SÃO PAULO,354100005001,Boqueirão,5,"POLYGON ((-46.41326 -24.00514, -46.41484 -24.0..."


Como estamos utilizando apenas dados de São Paulo, vamos seleciona-los

In [52]:
df_geo = df_geo[df_geo.NM_MUNICIP == "SÃO PAULO"]
df_geo.sample(3)

,ID,CD_GEOCODI,TIPO,CD_GEOCODS,NM_SUBDIST,CD_GEOCODD,NM_DISTRIT,CD_GEOCODM,NM_MUNICIP,NM_MICRO,NM_MESO,CD_GEOCODB,NM_BAIRRO,ID1,geometry
50136,115704.0,355030835000138,URBANO,35503083500,None,355030835,ITAIM BIBI,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,50137,"POLYGON ((-46.68735 -23.62244, -46.68742 -23.6..."
54797,120306.0,355030858000074,URBANO,35503085800,None,355030858,PEDREIRA,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,54798,"POLYGON ((-46.66075 -23.69040, -46.66016 -23.6..."
56253,121821.0,355030865000152,URBANO,35503086500,None,355030865,RAPOSO TAVARES,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,56254,"POLYGON ((-46.78750 -23.59962, -46.78799 -23.5..."


## Mesclando os dados

Com os dados devidamente tratados, vamos mescla-los para que a analise seja mais profunda. Os dataframes tratados são:

* df_imovel
* df_setor
* df_endereco
* df_geo

Vamos começar com os dados referentes aos endereços (df_endereço) e os dados dos valores dos imovéis (df_imove)

In [53]:
df_imovel.head()

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float,apenas_rua
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,Planalto Paulista,7000000.0,avenida itacira
2,Rua Alba Valdez,São Paulo,125,4,3,2,Jardim Reimberg,380000.0,rua alba valdez
3,NaN,São Paulo,310,3,2,4,Jardim Morumbi,685000.0,NaN
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,Mooca,540000.0,rua tobias barreto
5,Rua Graham Bell,São Paulo,440,4,4,6,Santo Amaro,1980000.0,rua graham bell


In [54]:
df_endereco.head()

,tipo_logr,logr_nome,logr_compl,numero,bairro,nome_localidade,sigla_uf,cep,latitude,longitude,rua
1,Rua,João Antônio Vasconcelos,Rua João Antônio Vasconcelos - até 627/628,627,Amizade,Araçatuba,SP,16074320.0,-21.190276,-50.448686,rua joão antônio vasconcelos
413269,Rua,A,Rua A,NaN,Jardim Glória,Americana,SP,13468220.0,-22.756531,-47.343738,rua a
413270,Rua,A,Rua A,NaN,Jardim São Roque,Americana,SP,13469414.0,-22.743734,-47.348631,rua a
413271,Rua,A,Rua A,NaN,Iate Clube de Campinas,Americana,SP,13475620.0,-22.737846,-47.333569,rua a
413272,Rua,João Bandini,Rua João Bandini,NaN,Jardim Miriam,Americana,SP,13469072.0,-22.744283,-47.351588,rua joão bandini


Realizando um left join, entre o 'df_imovel' e 'df_endereço', que retorna todos os registros do dataframe 'df_imovel' e os registros correspondentes do dataframe 'df_endereço' quando os valores em que os registros do campo 'apenas_rua' do dataframe 'df_imovel' for igual ao resgistro 'rua' do dataframe 'df_endereco'

In [55]:
df_imovel_endereco = pd.merge(
    left = df_imovel, 
    right = df_endereco[["rua", "cep", "latitude", "longitude"]], 
    how = "left", 
    left_on = "apenas_rua", 
    right_on = "rua"
).drop_duplicates(
    subset=df_imovel.columns
).query("cep > 0")

In [56]:
df_imovel_endereco

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float,apenas_rua,rua,cep,latitude,longitude
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,Planalto Paulista,7000000.0,avenida itacira,avenida itacira,11443160.0,-23.975993,-46.201317
5,Rua Alba Valdez,São Paulo,125,4,3,2,Jardim Reimberg,380000.0,rua alba valdez,rua alba valdez,4845200.0,-23.752990,-46.695889
6,NaN,São Paulo,310,3,2,4,Jardim Morumbi,685000.0,NaN,NaN,18125000.0,-23.531043,-47.255023
25,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,Mooca,540000.0,rua tobias barreto,rua tobias barreto,13474663.0,-22.698901,-47.294461
38,Rua Graham Bell,São Paulo,440,4,4,6,Santo Amaro,1980000.0,rua graham bell,rua graham bell,4737030.0,-23.593796,-46.760077
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89836,Avenida Odila,São Paulo,262,3,6,4,Planalto Paulista,1500000.0,avenida odila,avenida odila,4058020.0,-23.609093,-46.647258
89838,Rua Tapuçu,São Paulo,90,2,2,2,Vila Sofia,665000.0,rua tapuçu,rua tapuçu,4671185.0,-23.661018,-46.691521
89839,Rua Guararema,São Paulo,600,8,7,5,Bosque da Saúde,2300000.0,rua guararema,rua guararema,13030640.0,-22.922749,-47.076960
89856,Rua Estero Belaco,São Paulo,200,3,3,6,Vila da Saúde,1050000.0,rua estero belaco,rua estero belaco,4145020.0,-23.616124,-46.632983


Proximo passo será mesclar os dados do dataframe df_imovel_endereco (dataframe retornado da combinação entre df_imovel e df_endereço) com os dados do dataframe df_geo

In [57]:
df_geo.sample(3)

,ID,CD_GEOCODI,TIPO,CD_GEOCODS,NM_SUBDIST,CD_GEOCODD,NM_DISTRIT,CD_GEOCODM,NM_MUNICIP,NM_MICRO,NM_MESO,CD_GEOCODB,NM_BAIRRO,ID1,geometry
53219,118713.0,355030847000065,URBANO,35503084700,None,355030847,JOSÉ BONIFÁCIO,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,53220,"POLYGON ((-46.43980 -23.55457, -46.43994 -23.5..."
61824,127302.0,355030894000053,URBANO,35503089400,None,355030894,VILA SÔNIA,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,61825,"POLYGON ((-46.72951 -23.60892, -46.72952 -23.6..."
47416,112984.0,355030824000084,URBANO,35503082400,None,355030824,CIDADE LIDER,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,47417,"POLYGON ((-46.50118 -23.56023, -46.50130 -23.5..."


Criando um campo denominado 'Point' para armazenar a localização geografica (longitude e latitude) do imovél a venda.

In [58]:
df_imovel_endereco["Point"] = ""
for i in df_imovel_endereco.index:
    df_imovel_endereco["Point"][i] = Point(
        df_imovel_endereco["longitude"][i], 
        df_imovel_endereco["latitude"][i]
    )

/tmp/ipykernel_31747/1451948430.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imovel_endereco["Point"][i] = Point(


In [59]:
df_imovel_endereco.sample(5)

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float,apenas_rua,rua,cep,latitude,longitude,Point
85823,Rua José Figliolini,São Paulo,200,4,3,2,Vila Nilo,620000.0,rua josé figliolini,rua josé figliolini,2278020.0,-23.460072,-46.577147,POINT (-46.5771472 -23.4600718)
49651,NaN,São Paulo,148,3,4,2,Jardim Petrópolis,1200000.0,NaN,NaN,18125000.0,-23.531043,-47.255023,POINT (-47.2550228 -23.5310431)
76937,Rua Alessandro Manzoni,São Paulo,140,4,4,6,Vila Arriete,1000000.0,rua alessandro manzoni,rua alessandro manzoni,4445050.0,-23.681716,-46.685403,POINT (-46.6854026 -23.6817158)
27784,Rua Machado de Assis,São Paulo,285,3,5,4,Vila Mariana,2150000.0,rua machado de assis,rua machado de assis,13465581.0,-22.742070,-47.343600,POINT (-47.3436003 -22.7420696)
86738,Rua També,São Paulo,461,3,6,4,Jardim Guedala,3895000.0,rua també,rua també,7272370.0,-23.468030,-46.423202,POINT (-46.4232023 -23.4680304)


Criando um campo denominado setor_censo com o poligono na qual o ponto do imovel se localiza de acordo com os dados geoespaciais do IBGE

In [60]:
df_imovel_endereco['setor_censo'] = df_imovel_endereco["Point"].map(
    lambda x: df_geo.loc[df_geo.contains(x), 'CD_GEOCODI'].values
).str[0]

In [61]:
df_imovel_endereco.sample(3)

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float,apenas_rua,rua,cep,latitude,longitude,Point,setor_censo
13044,NaN,São Paulo,351,4,4,3,Lar São Paulo,1650000.0,NaN,NaN,18125000.0,-23.531043,-47.255023,POINT (-47.2550228 -23.5310431),NaN
43149,Rua Simão Dias da Fonseca,São Paulo,176,3,1,2,Cambuci,1500000.0,rua simão dias da fonseca,rua simão dias da fonseca,1539020.0,-23.572562,-46.622098,POINT (-46.6220976 -23.5725621),355030814000038
601,NaN,São Paulo,354,4,6,4,Jardim Guedala,2950000.0,NaN,NaN,18125000.0,-23.531043,-47.255023,POINT (-47.2550228 -23.5310431),NaN


E por fim, apos mesclar os dataframes df_imovel, df_endereco e df_geo, vamos mesclar o dataframe retornado com o df_setor

In [62]:
df_setor.sample(3)

,Cod_setor,Cod_Grandes Regiï¿½es,Nome_Grande_Regiao,Cod_UF,Nome_da_UF,Cod_meso,Nome_da_meso,Cod_micro,Nome_da_micro,Cod_RM,...,V003,V004,V005,V006,V007,V008,V009,V010,V011,V012
13330,355030869000102,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,1.81,1.30,2823.25,9457990.91,3138.70,9525031.01,2168.80,7081095.62,2596.45,7368001.25
1471,355030813000010,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,3.25,2.41,1357.01,1796352.66,1536.61,1757983.11,827.83,1214059.34,1264.74,1302390.34
3149,355030821000038,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,3.10,2.76,1776.80,5665743.89,2096.63,6017369.58,1340.38,3175852.10,1765.37,3433247.72


In [63]:
df_imovel_endereco["setor_censo"] = df_imovel_endereco["setor_censo"].astype(float)

In [64]:
df_imovel_endereco["setor_censo"]

0                 NaN
5        3.550308e+14
6                 NaN
25                NaN
38       3.550309e+14
             ...     
89836    3.550309e+14
89838    3.550308e+14
89839             NaN
89856    3.550309e+14
89857    3.550309e+14
Name: setor_censo, Length: 6522, dtype: float64

In [65]:
df_imovel_endereco_setor = pd.merge(
    left = df_imovel_endereco, 
    right = df_setor, 
    how = "left", 
    left_on = "setor_censo", 
    right_on = "Cod_setor"
)

In [66]:
df_imovel_endereco_setor.head()

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float,apenas_rua,rua,...,V003,V004,V005,V006,V007,V008,V009,V010,V011,V012
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,Planalto Paulista,7000000.0,avenida itacira,avenida itacira,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Rua Alba Valdez,São Paulo,125,4,3,2,Jardim Reimberg,380000.0,rua alba valdez,rua alba valdez,...,3.47,3.06,805.13,769188.78,1011.75,757433.42,518.77,426672.78,858.51,414378.88
2,NaN,São Paulo,310,3,2,4,Jardim Morumbi,685000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,Mooca,540000.0,rua tobias barreto,rua tobias barreto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rua Graham Bell,São Paulo,440,4,4,6,Santo Amaro,1980000.0,rua graham bell,rua graham bell,...,3.51,2.31,1026.36,272168.67,1143.96,167733.64,735.67,367088.08,1086.94,159622.46


In [67]:
df_imovel_endereco_setor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6522 entries, 0 to 6521
Data columns (total 48 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rua                    4367 non-null   object 
 1   cidade                 6522 non-null   object 
 2   metragem               6522 non-null   int64  
 3   quartos                6522 non-null   int64  
 4   banheiros              6522 non-null   int64  
 5   vagas                  6522 non-null   int64  
 6   bairro                 6522 non-null   object 
 7   preco_anuncio_float    6522 non-null   float64
 8   apenas_rua             4367 non-null   object 
 9   rua                    4367 non-null   object 
 10  cep                    6522 non-null   float64
 11  latitude               6522 non-null   float64
 12  longitude              6522 non-null   float64
 13  Point                  6522 non-null   object 
 14  setor_censo            3019 non-null   float64
 15  Cod_

In [68]:
df_imovel_endereco_setor.to_csv('dados_imoveis_treatment.csv')